In [8]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, DatePicker
import matplotlib.dates as mdates
from datetime import datetime
from matplotlib.dates import DateFormatter

# Функция для получения данных с ЦБ РФ за определённый период
def get_currency_data(start_date, end_date):
    url = f'https://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={start_date}&date_req2={end_date}&VAL_NM_RQ=R01235'
    response = requests.get(url)
    tree = ET.ElementTree(ET.fromstring(response.content))
    root = tree.getroot()

    data = []
    for record in root.findall('Record'):
        date = record.attrib['Date']
        value = float(record.find('Value').text.replace(',', '.'))
        data.append([date, value])

    df = pd.DataFrame(data, columns=['Date', 'USD/RUB'])
    df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')
    df.set_index('Date', inplace=True)
    return df

# Задаем диапазон дат для получения исторических данных
start_date = '01/01/2023'  # формат: дд/мм/гггг
end_date = datetime.now().strftime('%d/%m/%Y')   # формат: дд/мм/гггг

# Загружаем данные за указанный период только один раз
df = get_currency_data(start_date, end_date)


# Функция для построения графика
def plot_currency(start_date, end_date):
    plt.clf()  # Очищаем фигуру перед перерисовкой
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Фильтрация данных в зависимости от ползунка
    df_filtered = df.loc[start_date:end_date]
    
    ax.plot(df_filtered.index, df_filtered['USD/RUB'], marker='o', color='blue')
    
    # Форматирование графика
    ax.set_title('Курс USD к RUB по данным ЦБ РФ')
    ax.set_xlabel('Дата')
    ax.set_ylabel('Курс')
    ax.grid(True)

    # Настраиваем формат отображения дат на оси X
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))  # Отображаем каждую неделю
    ax.xaxis.set_major_formatter(DateFormatter("%d-%m-%Y"))
    plt.xticks(rotation=45, ha='right')

    plt.show()

# Создаем ползунки для выбора диапазона дат
date_start_picker = DatePicker(description='Дата начала', value=pd.to_datetime(start_date))
date_end_picker = DatePicker(description='Дата конца', value=pd.to_datetime(end_date))

interact(plot_currency, start_date=date_start_picker, end_date=date_end_picker);



C:\Users\vasja\AppData\Local\Temp\ipykernel_20540\1409230493.py:62: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  date_end_picker = DatePicker(description='Дата конца', value=pd.to_datetime(end_date))


interactive(children=(DatePicker(value=Timestamp('2023-01-01 00:00:00'), description='Дата начала'), DatePicke…